# Lecture 11.1: Statistical Modeling
<div style="border: 1px double black; padding: 10px; margin: 10px">
From now until the end of the course we will focus on statistical modeling ([Part IV] of your book).

    




In [1]:
library(tidyverse)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.2     ✔ purrr   0.3.4
✔ tibble  3.0.3     ✔ dplyr   1.0.2
✔ tidyr   1.1.1     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.5.0

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()



## What is a model
A statistical model is a mathematical formula that relates an outcome with one or more explanatory variables.

$$\underbrace{Y}_{\text{outcome}} = \underbrace{f}_{\text{model function}}(\underbrace{X}_{\text{explainer}}) + 
    \underbrace{\epsilon}_\text{noise}$$

### Model classes
The types of functions $f$ that we allow determine what is called the *model class*. For example, in STATS 250 you learned about linear regression, where $f$ is any function of the form 

$$f(x) = a_1 + a_2x $$

for some *parameters* $a_1$ and $a_2$. This defines a whole *family* of models: one for each choice of slope and intercept.

### Model fitting
The process of *fitting* a model refers to selecting the particular choice $\hat{a},\hat{b}$ from the family of models that we have chosen, in order to best fit the data. The fitted model is the member of the model family we have selected that is "closest" to the data. This *does not* mean that this is the "true" model! In most cases there is no "true" model. The goal of a model is not to uncover truth, but to discover a simple approximation that is still useful.

### Model selection
There is also the question of which family of models to use. In other words, which types of functions $f(x)$ to use. To use a fashionable example, we could have instead chosen our model family to be 

$$\{f: \text{$f$ is a neural network}\}.$$ 

The problem of choosing a model family is known as *model selection*. It is a much trickier problem than model fitting because there is no one correct answer: "all models are wrong"; the appropriate model family balances our needs for interpretability, predictiveness, etc.

### Example
We'll use the `modelr` package (again part of tidyverse) to learn about modeling.

The `modelr` package comes with a simple bivariate dataset that we can model:

The first step is to explore the data using the techniques we have already learned:

There is a strong linear relationship. We suspect that a good model might be the one we saw above: $y = a_1 + a_2x$. If we select a particular $a$ and $b$, this gives us a potential model for the data. We can plot this for various choices of $a_1$ and $a_2$ and see visually see how well it might fit:

The red line represents the value of $y$ that we would predict for each value of $x$. To measure how good our model fit is we can do the following: for each pair of data points $(x_i,y_i)$, measure the distance $|\hat{y}_i - y_i|$ between predicted and observed values of $y_i$. The value $\hat{y}_i - y_i$ is called the *residual*. It's the component of the data that isn't predicted by our model. Adding up the residuals gives us a measure of how good our model fits the data. If we predict the data perfectly ($\hat{y}_i = y_i$ for all $i$) then this would equal zero, so lower values are better. (Later you will learn that this is only true up to a point; it is generally not a good idea to fit the data perfectly.)

Let's define a model from this class:

Now we will write a function which takes a model and plots the residuals for each data point:

(I have jittered the data slightly to make the regression effect more apparent.)

To measure how well our model peforms we define a function which adds up the square of the residuals:

Now we want to figure out what is the best fitting model, i.e. the choice of $a$ that minimizes `measure_distance` as defined above. Let's first try simply generating a bunch of random models and seeing which one has the smallest `measure_distance`:

Let's look at the three best-fitting models (the three indices $i$ for which `sim1_dist(a1[i], a2[i])` is smallest).

We can also visualize the model fits by plotting `dist[i]` for each pair `c(a1[i], a2[i])`:

Rather than exploring the models randomly let us systematically try all points on a grid of values:

If we repeat the same plot, we see that the best fitting models are roughly centered around `a=c(4,2)`.

If we plot these models they all fit the data pretty well:

To find the "best" model you could imagine taking a finer and finer grid of points:

But this "brute force" approach is wasteful. We can do much better by using an optimization algorithm to find the minimum for us.

### Optimization in R
Optimization means "find the minimum of a function". In R the command `optim` can be used to optimize a function:

We can use `optim` to find the values `a1` and `a2` that minimize `sim1_dist`: 

### The `lm` command
The simple linear model class we are considering is a special case of the larger family

$$y = a_0 + a_1 x_1 + \dots + a_p x_p.$$

R has a special commmand `lm` used to fit this type of model:

Note that these are the same parameter estimates that we got using `optim`:

## Visualizing models
In this section we will learn some ways to visualize statistical models. One way is to look at the *predictions* made by the model over the observed range of values for our explanatory variable(s). The command `modelr::data_grid` will generate this for us:

In [32]:
sim1 %>% print 

# A tibble: 30 x 2
       x     y
   <int> <dbl>
 1     1  4.20
 2     1  7.51
 3     1  2.13
 4     2  8.99
 5     2 10.2 
 6     2 11.3 
 7     3  7.36
 8     3 10.5 
 9     3 10.5 
10     4 12.4 
# … with 20 more rows


Next we will add the predictions for each $x$ value in `grid`:v

Finally, we plot these predictions using `ggplot`:

What is the advantage of using `modelr` here versus just extracting the coefficients and doing it ourselves, as before? The `modelr` code works with any model. We could have used something more complicated for `sim1_mod`:

### Residuals
Each data point $y_i$ can be decomposed as 

$$y_i = \underbrace{\hat{y}_i}_\text{prediction} + \underbrace{\epsilon_i}_\text{residual}.$$

The prediction is the pattern in the data that the model has captured, and the residual is what is left over. To explore the residuals, use `add_residuals()`.

If you recall from STATS 250, the linear model assumes that the residuals $\epsilon_i$ are independent and normally distributed. By visualizing the residuals, we may judge whether this assumption holds or not:

If the model has done a good job of capturing patterns in the data, then the residuals should look like random noise. (In other words, if the residuals contain obvious patterns, then there is more modeling work to be done!) You should confirm this by visualizing the residuals:

## Formulas
We have seen examples of formulas when working with the `facet_*()` commands, and also used them in place of anonymous functions with `map`. Formulas are most commonly used in model fitting commands like `lm`. 

We saw that the notation `y ~ x` cause R to fit the model 

$$y = a_1 + a_2 x.$$

The command `model_matrix` can show us exactly how formulas work on specific data examples:

In [2]:
 df <- tribble(
   ~y, ~x1, ~x2,
   4, 2, 5,
   5, 1, 6,
   1, 2, 3
 ) %>% print


# A tibble: 3 x 3
      y    x1    x2
  <dbl> <dbl> <dbl>
1     4     2     5
2     5     1     6
3     1     2     3


By default, R will add an intercept term. If you want to fit a model with no intercept, you should subtract 1 from the formula: